# Mixed Beverage Gross Receipts

This file contains a list of taxpayers required to report mixed beverage gross receipts tax reports under Tax Code Chapter 183, Subchapter B. The list provides taxpayer names, amounts reported, and other public information.

https://data.texas.gov/dataset/Mixed-Beverage-Gross-Receipts/naix-2893

In [1]:
from datetime import datetime

import altair as alt
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 50)

In [2]:
TAX_CUTOVER = datetime(2014, 1, 1)

Load original gzipped csv and re-save as a parquet file for faster loads

In [3]:
# df = pd.read_csv('Mixed_Beverage_Gross_Receipts.csv.gz', dtype={'Taxpayer Zip': str})
# df.to_parquet('receipts.parquet')

Obligation End Date = Last Day of Reporting Period

Total Receipts = Liquor + Wine + Beer + Cover Charge. To calculate the tax due, multiply the total receipts by .067 if reporting period is January 1, 2014 or greater and by .14 if reporting period is prior to January 1, 2014.

In [4]:
df = pd.read_parquet('receipts.parquet')
df['last_day_reporting_period'] = pd.to_datetime(df['Obligation End Date'])

print(f'total rows: {len(df):,}')
df.head(2)

total rows: 2,748,503


,Taxpayer Number,Taxpayer Name,Taxpayer Address,Taxpayer City,Taxpayer State,Taxpayer Zip,Taxpayer County,Location Number,Location Name,Location Address,Location City,Location State,Location Zip,Location County,Inside/Outside City Limits,TABC Permit Number,Responsibility Begin Date,Responsibility End Date,Obligation End Date,Liquor Receipts,Wine Receipts,Beer Receipts,Cover Charge Receipts,Total Receipts,last_day_reporting_period
0,32047970895,HONDURAS MAYA CAFE & BAR LLC,8011 HAZEN ST,HOUSTON,TX,77036,101,1,HONDURAS MAYA CAFE & BAR LLC,5945 BELLAIRE BLVD STE B,HOUSTON,TX,77081,101,Y,MB817033,08/16/2012,09/12/2019,07/31/2019,0,0,0,0,0,2019-07-31
1,32049923835,"MERMAID KARAOKE PRIVATE CLUB, INC.",2639 WALNUT HILL LN STE 225,DALLAS,TX,75229,57,1,MERMAID KARAOKE PRIVATE CLUB,1310 W CAMPBELL RD STE 103,RICHARDSON,TX,75080,57,Y,N 837378,04/12/2013,07/01/2015,08/31/2014,480,185,1374,0,2039,2014-08-31


Group by on taxpayer number

In [5]:
print(f"  Total taxpayer names: {len(df['Taxpayer Number'].unique()):,}")
print(f"Total taxpayer numbers: {len(df['Taxpayer Name'].unique()):,}")

  Total taxpayer names: 34,409
Total taxpayer numbers: 34,357


## Taxes

In [6]:
def calc_tax(row: pd.Series) -> float:
    reporting_period, total = row
    pct = 0.14 if reporting_period < TAX_CUTOVER else 0.067
    return total * pct

In [7]:
print(f"Receipts from {df['last_day_reporting_period'].min():%b %d, %Y} to {df['last_day_reporting_period'].max():%b %d, %Y}")

Receipts from Jan 31, 2007 to Jun 30, 2022


In [8]:
receipt_totals = df.groupby(['Taxpayer Number', 'last_day_reporting_period'])['Total Receipts'].sum().reset_index()
receipt_totals['year'] = receipt_totals['last_day_reporting_period'].dt.year

receipt_totals['old_taxes'] = receipt_totals['Total Receipts'] * 0.14
receipt_totals['new_taxes'] = receipt_totals['Total Receipts'] * 0.067
receipt_totals['tax_diff'] = receipt_totals['old_taxes'] - receipt_totals['new_taxes']
receipt_totals['taxes_due'] = np.where(receipt_totals['last_day_reporting_period'] < TAX_CUTOVER, receipt_totals['old_taxes'], receipt_totals['new_taxes'])

receipt_totals.head(2)

,Taxpayer Number,last_day_reporting_period,Total Receipts,year,old_taxes,new_taxes,tax_diff,taxes_due
0,10105559610,2007-02-28,19084,2007,2671.76,1278.628,1393.132,2671.76
1,10105559610,2007-03-31,31599,2007,4423.86,2117.133,2306.727,4423.86


In [9]:
print(f"       Total receipts: {receipt_totals['Total Receipts'].sum():,.2f}")
print(f"Total taxes collected: {receipt_totals['taxes_due'].sum():,.2f}")

       Total receipts: 89,114,473,503.00
Total taxes collected: 8,363,280,716.26


## Top grossing companies

Out of the top 1,000 Total Receipts over all reporting periods

In [10]:
(pd.merge(receipt_totals.sort_values('Total Receipts', ascending=False).head(1_000),
          df[['Taxpayer Number', 'Taxpayer Name']],
          on='Taxpayer Number')['Taxpayer Name'].drop_duplicates())

0                              CHILI'S BEVERAGE COMPANY, INC.
7125584     ARAMARK SPORTS AND ENTERTAINMENT SERVICES OF T...
7137354                              PAPPAS RESTAURANTS, INC.
9086028                              LEGENDS HOSPITALITY, LLC
9115015                             LEVY TEXAS BEVERAGES, LLC
9115024                       HOSPITALITY INTERNATIONAL, INC.
9942312                           ROADHOUSE ENTERPRISES, INC.
10297167                                   BLAZIN WINGS, INC.
10824525                     LEVY PREMIUM FOODSERVICE, L.L.C.
10878561                                  SC BEVERAGE COMPANY
11010277                            PERRY'S RESTAURANTS, LTD.
11031760                            OL BEVERAGE HOLDINGS, LLC
11037850                            TW RESTAURANT HOLDER, LLC
11373325    ARAMARK SPORTS ENT SVC'S & LEVY PREM FOOD SVC LLC
11374613                                      SALTGRASS, INC.
11625217                                 WLS BEVERAGE COMPANY
11630365

### group by year

In [11]:
totals_by_year = receipt_totals.groupby('year')[['Total Receipts', 'old_taxes', 'new_taxes', 'tax_diff', 'taxes_due']].sum().reset_index()
totals_by_year['taxes_due_str'] = totals_by_year['taxes_due'].apply(lambda i: f'{i / 1_000_000:.1f} M')
totals_by_year.style.format('{:,.0f}', subset=['Total Receipts', 'old_taxes', 'new_taxes', 'tax_diff', 'taxes_due'])

,year,Total Receipts,old_taxes,new_taxes,tax_diff,taxes_due,taxes_due_str
0,2007,"4,032,838,388","564,597,374","270,200,172","294,397,202","564,597,374",564.6 M
1,2008,"4,267,790,879","597,490,723","285,941,989","311,548,734","597,490,723",597.5 M
2,2009,"4,241,102,855","593,754,400","284,153,891","309,600,508","593,754,400",593.8 M
3,2010,"4,489,264,088","628,496,972","300,780,694","327,716,278","628,496,972",628.5 M
4,2011,"4,884,168,070","683,783,530","327,239,261","356,544,269","683,783,530",683.8 M
5,2012,"5,291,161,829","740,762,656","354,507,843","386,254,814","740,762,656",740.8 M
6,2013,"5,569,166,926","779,683,370","373,134,184","406,549,186","779,683,370",779.7 M
7,2014,"5,914,342,099","828,007,894","396,260,921","431,746,973","396,260,921",396.3 M
8,2015,"6,215,559,828","870,178,376","416,442,508","453,735,867","416,442,508",416.4 M
9,2016,"6,471,401,208","905,996,169","433,583,881","472,412,288","433,583,881",433.6 M


### Total Receipts by year

In [12]:
alt.Chart(totals_by_year).mark_bar().encode(x='year:O', y='Total Receipts', tooltip=['taxes_due_str'])

alt.Chart(...)

### taxes due by year

In [13]:
alt.Chart(totals_by_year).mark_bar().encode(x='year:O', y='taxes_due')

alt.Chart(...)

### totals for the whole data set

In [14]:
(receipt_totals.set_index(['Taxpayer Number', 'last_day_reporting_period'])
               .sum().reset_index().set_index('index')
               .style.format('{:,.2f}'))

,0
index,
Total Receipts,"89,114,473,503.00"
year,"4,376,644,973.00"
old_taxes,"12,476,026,290.42"
new_taxes,"5,970,669,724.70"
tax_diff,"6,505,356,565.72"
taxes_due,"8,363,280,716.26"


### Before Jan 1, 2014

In [15]:
m = (receipt_totals['last_day_reporting_period'] < TAX_CUTOVER)
(receipt_totals[m].set_index(['Taxpayer Number', 'last_day_reporting_period'])
                  .sum().reset_index().set_index('index')
                  .style.format('{:,.2f}'))

,0
index,
Total Receipts,"32,775,493,035.00"
year,"1,782,309,695.00"
old_taxes,"4,588,569,024.90"
new_taxes,"2,195,958,033.35"
tax_diff,"2,392,610,991.55"
taxes_due,"4,588,569,024.90"


### on and after Jan 1, 2014

In [16]:
m = (receipt_totals['last_day_reporting_period'] >= TAX_CUTOVER)
(receipt_totals[m].set_index(['Taxpayer Number', 'last_day_reporting_period'])
                  .sum().reset_index().set_index('index')
                  .style.format('{:,.2f}'))

,0
index,
Total Receipts,"56,338,980,468.00"
year,"2,594,335,278.00"
old_taxes,"7,887,457,265.52"
new_taxes,"3,774,711,691.36"
tax_diff,"4,112,745,574.16"
taxes_due,"3,774,711,691.36"
